In [1]:
API_KEY = open("key.txt", "r").read().replace("\n","")

In [2]:
# as per https://developers.google.com/places/web-service/search
MAX_RADIUS      = 50000  #in meters
MAX_RESULT_SIZE = 20
MAX_PAGE_COUNT  = 3
DEFAULT_SLEEP   = 2
THREAD_COUNT    = 16
MAX_JOBS        = 85 # 1 + 4 + 16 + 64 

In [3]:
import requests
import pandas as pd 
from pandas.io.json import json_normalize
import json
import time
import numpy as np
import queue
import threading
import datetime
import geopy
import geopy.distance
from geopy.distance import VincentyDistance

%matplotlib inline

In [4]:
city_centers ={ "Riyadh":(24.713600, 46.675300),
                "Jeddah":(21.485800, 39.192500),
                "Makkah": (21.389100, 39.857900),
                "Madinah": (24.524700, 39.569200),
                "Dammam":(26.420700, 50.088800),
                "Khobar":(26.217200, 50.197100)}

In [5]:
completed_jobs = []
enqueued = []
dfs = []
job_queue = queue.Queue()

In [6]:
#return centers and radius of minimal circles containing quarters of a given square defined by its corners
def get_smaller_circles(location,radius):
# given: lat1, lon1, b = bearing in degrees, d = distance in kilometers
    origin = geopy.Point(location)
    origins=[]
    radius=radius*2**0.5/2
    for i in range(4):
        destination = geopy.distance.geodesic(meters=radius).destination(origin, 90*i+45)
        origins.append((destination.latitude, destination.longitude))
    return origins,radius


In [7]:
def add_job(paramters,job_queue):
    print("adding job location {} radius {}".format(paramters['location'],paramters['radius']))
    enqueued.append(paramters)
    job_queue.put(paramters)

def get_job(job_queue):
    return job_queue.get()

In [8]:
def find_place_from_text(query,location,radius=MAX_RADIUS,api_key=API_KEY,sleep_time=DEFAULT_SLEEP,start_time=None,start_location=None):
    
    results = []
    next_page_token = True
    next_page_count = 0
    
    while next_page_token:
    
        #1 you need to put your google key 
        # https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Museum%20of%20Contemporary%20Art%20Australia&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key=YOUR_API_KEY
        # query = urlencode.encode('Musuem Contemporty')
        if len(results)==0:
            next_token = ''
        else:
            next_token = '&pagetoken=' + next_page_token
            next_page_count+=1

        #fields = ['photos','formatted_address','name','opening_hours','rating']
        
        result = requests.get("https://maps.googleapis.com/maps/api/place/textsearch/json?query={query_text}&inputtype=textquery&key={query_api_key}&location={query_location}&radius={query_radius}&rank_by={query_rank_by}&type={query_type}{next_token}".format(\
            query_api_key=api_key,
            query_text= query,
            query_location=','.join([str(x) for x in location]),
            query_radius=radius,
            query_rank_by="distance",
            query_type=query,
            next_token=next_token
        ))
        

        # turn response to json
        google_result = result.json()
        #collect next page token
        next_page_token = google_result.get('next_page_token', None)

        # get results field
        #results = google_result["results"]
        # update results count
        
        # make df from results
        results.append(pd.DataFrame.from_dict(google_result["results"], orient='columns'))

        # add time delay because google is stupid
        
        if(next_page_token):
            time.sleep(sleep_time)
    df=pd.concat([*results]).reset_index(drop=True)   
    return df ,next_page_count

In [9]:
df, count =find_place_from_text("bla",city_centers['Riyadh'],radius=100,api_key=API_KEY,sleep_time=DEFAULT_SLEEP)
df

,formatted_address,geometry,icon,id,name,photos,place_id,rating,reference,types,user_ratings_total
0,"Bla, Mali","{'location': {'lat': 12.9549865, 'lng': -5.756...",https://maps.gstatic.com/mapfiles/place_api/ic...,5c4dbf6ced8fd7ea7ef5e7ce90d05dc2624c55ee,Bla,"[{'height': 2976, 'html_attributions': ['<a hr...",ChIJO_nVb773SA4R06rLaftaofo,0,ChIJO_nVb773SA4R06rLaftaofo,"[locality, political]",0


In [11]:
print(df.geometry[0])
geopy.distance.geodesic(city_centers['Riyadh'],(12.9549865,-5.7563151)).km

{'location': {'lat': 12.9549865, 'lng': -5.7563151}, 'viewport': {'northeast': {'lat': 12.9650815, 'lng': -5.7385541}, 'southwest': {'lat': 12.9374363, 'lng': -5.7727574}}}


5643.147674666189

In [12]:
def worker():
    #To end threads restart kernal. Careful: export your resutls into a csv file
    while True:
        item = get_job(job_queue)
        if item is None:
            break
        df,next_page_count = find_place_from_text(**item)
        size=len(df)
        item["compeleted"]=datetime.datetime.now()
        item["size"]=size
        item["next_page_count"]=next_page_count
        item["distance_from_start_point"]=geopy.distance.geodesic(item['start_location'],item['location']).m
        completed_jobs.append(item)
        
        if(size==MAX_PAGE_COUNT*MAX_RESULT_SIZE and len(enqueued)<MAX_JOBS):
            origins,radius = get_smaller_circles(item['location'],item['radius'])
            for origin in origins:
                job={'query':item['query'],
                'location': origin,
                'radius':radius,
                'api_key':item['api_key'],
                'start_time':datetime.datetime.now(),
                'start_location': item['start_location'],
                'sleep_time':item['sleep_time']}
                add_job(job,job_queue)
            
        dfs.append(df)
        #q.task_done()


threads = []
for i in range(THREAD_COUNT):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)

In [13]:
job={'query':'hospital',
     'location': city_centers['Riyadh'],
     'radius':MAX_RADIUS,
     'api_key':API_KEY,
     'start_time':datetime.datetime.now(),
     'start_location': city_centers['Riyadh'],
     'sleep_time':DEFAULT_SLEEP}
add_job(job,job_queue)

adding job location (24.7136, 46.6753) radius 50000
adding job location (24.939087849221874, 46.92282473604873) radius 35355.33905932738
adding job location (24.487698449538453, 46.92193381732826) radius 35355.33905932738
adding job location (24.48769844953845, 46.42866618267173) radius 35355.33905932738
adding job location (24.939087849221877, 46.42777526395127) radius 35355.33905932738
adding job location (24.647191465825493, 46.60328364489524) radius 25000.000000000004
adding job location (24.32800070691886, 46.60284358904809) radius 25000.000000000004
adding job location (24.32800070691886, 46.254488776295375) radius 25000.000000000004
adding job location (24.647191465825497, 46.254048720448225) radius 25000.000000000004
adding job location (24.647191465825497, 47.09655127955177) radius 25000.000000000004
adding job location (24.328000706918864, 47.09611122370461) radius 25000.000000000004
adding job location (24.32800070691886, 46.7477564109519) radius 25000.000000000004
adding jo

In [14]:
len(completed_jobs)

85

In [15]:
hospitals = pd.concat([*dfs]).reset_index(drop=True) 

In [16]:
len(hospitals),len(hospitals.name.unique())

(5100, 411)

In [17]:
hospitals.head()

,formatted_address,geometry,icon,id,name,opening_hours,photos,place_id,plus_code,rating,reference,types,user_ratings_total
0,"2565 Street Number 74, Al Olaya، العليا، الريا...","{'location': {'lat': 24.7039701, 'lng': 46.677...",https://maps.gstatic.com/mapfiles/place_api/ic...,54f84e985ae80c6fc4228cfbcf164edeec41c318,مستشفى د.سليمان الحبيب للنساء والولادة,{'open_now': True},"[{'height': 1836, 'html_attributions': ['<a hr...",ChIJD6tHHCkDLz4RZ9lVUWrFsVk,"{'compound_code': 'PM3H+H3 العليا، الرياض', 'g...",3.7,ChIJD6tHHCkDLz4RZ9lVUWrFsVk,"[hospital, point_of_interest, establishment]",23
1,شارع عنيزة، Ar Rawabi، الرياض 14214,"{'location': {'lat': 24.7024362, 'lng': 46.779...",https://maps.gstatic.com/mapfiles/place_api/ic...,c5ae023b383055c13f2e0b840d03ee455cf29e92,مستشفى رعاية الرياض,{'open_now': True},"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJmTF0gdYGLz4RnJerk0cryoI,"{'compound_code': 'PQ2H+XP الروابي، الرياض', '...",3.3,ChIJmTF0gdYGLz4RnJerk0cryoI,"[hospital, point_of_interest, establishment]",427
2,"King Fahd Road, Al Olaya، الرياض 12214","{'location': {'lat': 24.7043533, 'lng': 46.676...",https://maps.gstatic.com/mapfiles/place_api/ic...,a2690407aaf08060a208448aff746bece6ddc39b,مستشفى الدكتور سليمان الحبيب,{'open_now': True},"[{'height': 1334, 'html_attributions': ['<a hr...",ChIJ4YhtDCkDLz4RzwmoIa6kz_U,"{'compound_code': 'PM3G+PM العليا، الرياض', 'g...",3.3,ChIJ4YhtDCkDLz4RzwmoIa6kz_U,"[hospital, point_of_interest, establishment]",51
3,2712 شارع موسى بن نصير، العليا، الرياض 12214,"{'location': {'lat': 24.70352, 'lng': 46.67793...",https://maps.gstatic.com/mapfiles/place_api/ic...,f5e765ef9e3db59694dd4791bfdf7d7c17f034e8,مستشفى الدكتور سليمان الحبيب ( العظام ),{'open_now': True},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJv07EtygDLz4RI0_dF2k2yNY,"{'compound_code': 'PM3H+C5 العليا، الرياض', 'g...",3.3,ChIJv07EtygDLz4RI0_dF2k2yNY,"[hospital, point_of_interest, establishment]",23
4,"Dirab Branch Road, Namar، الرياض 14963","{'location': {'lat': 24.5606621, 'lng': 46.676...",https://maps.gstatic.com/mapfiles/place_api/ic...,a8fc04c67114f45a7b8da5da7871122532acf150,مستشفى دله - نمار,{'open_now': True},"[{'height': 3000, 'html_attributions': ['<a hr...",ChIJBzDJE9EPLz4RzE9_EPWzJJM,"{'compound_code': 'HM6G+7Q نمار، الرياض', 'glo...",3.9,ChIJBzDJE9EPLz4RzE9_EPWzJJM,"[hospital, point_of_interest, establishment]",93


In [18]:
hospitals.name.unique()

array(['مستشفى د.سليمان الحبيب للنساء والولادة', 'مستشفى رعاية الرياض',
       'مستشفى الدكتور سليمان الحبيب',
       'مستشفى الدكتور سليمان الحبيب ( العظام )', 'مستشفى دله - نمار',
       'Al Safwa Hospital مستشفى الصفوة', 'مستشفى الملك فهد',
       'المستشفى السعودي الألماني', 'Green Crescent Hospital',
       'مستشفى الأمير محمد بن عبدالعزيز',
       'مدينة الملك عبدالعزيز الطبية للحرس الوطني',
       'إدارة الشؤون الصحية بالحرس الوطني', 'مستشفى المواساة',
       'Dallah Children Hospital',
       'مركز الملك عبدالعزيز لأمراض وجراحة القلب',
       'العيادات الشمالية مستشفى دلة', 'Dallah Namar Hospital',
       'مدخل علم الاورام', 'مستشفى اليمامة', 'Aster Sanad Hospital',
       'مستشفى رعاية الوطني', 'مستشفى الملك فيصل التخصصي ومركز الأبحاث',
       'مستشفى الأطفال في مدينة الملك فهد الطبية',
       'مستشفى د. سليمان الحبيب في التخصصي',
       'مستشفى المركز التخصصي الطبي', 'د.سليمان الحبيب', 'مستشفى الحمادي',
       'المستشفى التخصصي الطبي البرج رقم 2', 'مستشفى الجزيرة',
       'مر